Copy your Cognitive Service Key (`cognitive_key`) and Service Region (`cognitive_service_region`) to the variables listed below.

In [2]:
cognitive_key = '<YourKey>'
cognitive_service_region = '<YourRegion>'

print('Cognitive services ready at {}'.format(cognitive_service_region))

Cognitive services ready at centralus


Let's install Azure Speech SDK

In [19]:
pip install azure.cognitiveservices.speech

  Using cached azure_cognitiveservices_speech-1.15.0-cp36-cp36m-manylinux1_x86_64.whl (3.1 MB)
Note: you may need to restart the kernel to use updated packages.


### Text Translation

We have to submit an HTTP request to use the Text Translation API, as there is no Python SDK for this service. We are submitting the parameters, headers, and the JSON body for our text to get back a JSON that contains all the translations.

In [8]:
import requests, uuid, json

endpoint = "https://api.cognitive.microsofttranslator.com/translate"

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['de', 'it']
}

headers = {
    'Ocp-Apim-Subscription-Key': cognitive_key,
    'Ocp-Apim-Subscription-Region': cognitive_service_region,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

body = [{
    'text': 'Hello World!'
}]

request = requests.post(endpoint, params=params, headers=headers, json=body)
response = request.json()

print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

[
    {
        "translations": [
            {
                "text": "Hallo Welt!",
                "to": "de"
            },
            {
                "text": "Salve, mondo!",
                "to": "it"
            }
        ]
    }
]


### Speech Translation

We are given a `speech-sample.wav` file with some speech in it that has to be converted to German and French. First, we have to create a **Translation Config** object that includes our service access key and region, the original recognition language,
and finally, the target languages that we want the speech to be converted. Once our **Translation Config** is ready, we are setting up an **Audio Config** that points out our `speech-sample.wav` file as the translation source. Finally, we are initializing a **Recognizer** 
by passing the two configuration objects to `speechsdk.translation.TranslationRecognizer`. Calling the `recognize_once()` gets us a recognized text and its translation to all languages we have requested.

In [3]:
import azure.cognitiveservices.speech as speechsdk

translation_config = speechsdk.translation.SpeechTranslationConfig(
        subscription=cognitive_key, region=cognitive_service_region,
        speech_recognition_language='en-US',
        target_languages=('de', 'fr'))
audio_config = speechsdk.audio.AudioConfig(filename="speech-sample.wav")

recognizer = speechsdk.translation.TranslationRecognizer(
    translation_config=translation_config, audio_config=audio_config)

result = recognizer.recognize_once()

if result.reason == speechsdk.ResultReason.TranslatedSpeech:
    print("""Recognized: {}
    German translation: {}
    French translation: {}""".format(
        result.text, result.translations['de'], result.translations['fr']))
elif result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized: {}".format(result.no_match_details))
elif result.reason == speechsdk.ResultReason.Canceled:
    print("Translation canceled: {}".format(result.cancellation_details.reason))
    if result.cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(result.cancellation_details.error_details))

Recognized: This is me talking to see how Azure Cognitive Services does speech recognition.
    German translation: Dies ist ein Gespräch, um zu sehen, wie Azure Cognitive Services Spracherkennung macht.
    French translation: C’est moi qui parle pour voir comment Azure Cognitive Services fait la reconnaissance vocale.


Good work!